In [ ]:
GPY
In this model, we tried to use GPY as we were getting LinAlg errors while using PMYC3
We tried to mix the Matern32 and RBF kernel in this case, as Matern32 can be used for long term trend prediction and RBF can fit 
in the actual data, so if RBF is able to predict the points, then Matern32 can pick the non seasonal trend.
We used Matern32 for prediction to capture the non seasonal trend

In [ ]:
import GPy
def run_model(X,Y,plot=False):    
    # define kernel
    bias = GPy.kern.Bias(1,variance=1.)
    ker1 = GPy.kern.Matern32(1)
    ker5 = GPy.kern.RBF(1) 

    # create a GP model
    model = GPy.models.GPRegression(X,Y,kernel=bias+ker1+ker5)


    # it finds the optimal parameters of the RBF kernel by maxisiming the marginal likelihood 
    model.optimize_restarts(5)# this maximise the marginal likelihood, with random restarts
    #we  plot it
    X_new = np.linspace(0, 1, 100)[:,None]
    f_pred, sigm2_pred = model.predict(X_new, kern=ker1)

    posterior_probability_deriviative_is_positive = len(np.where(np.gradient(f_pred[:,0],X_new[:,0])>0)[0])/len(f_pred[:,0])
    print("probability that the function is increasing =", posterior_probability_deriviative_is_positive)
    if posterior_probability_deriviative_is_positive>=0.95:
        return 1
    else:
        return 0

In [ ]:
test_df = pd.read_csv('/kaggle/input/sales-forecast-et5003-competition/test.csv')

In [ ]:
x = np.linspace(0,1,100)[:,None]
Decision = pd.DataFrame(columns=['Id','Category'])
# train_df.shape[0]
for r in range(test_df.shape[0]):
    id_row = test_df.iloc[r,0]
    y = test_df.iloc[r,1:101].values[:,None]
    decision = run_model(x,y)
    Decision = Decision.append({'Id': int(id_row), 'Category': int(decision)}, ignore_index=True) 

In [ ]:
This  model gave us ~70% accuracy on leader board